# Bristol - List

In [1]:
import sys
sys.path.insert(0, '../../../lib/')

import tabula
import numpy as np
import pandas as pd
import importlib

import pdfexport
importlib.reload(pdfexport)

from pdfexport import *

## Import PDF

In [2]:
# Read pdf into DataFrame
df = tabula.read_pdf("bms_SWITZ-2019-EFPIA-report.pdf", pages='all', lattice=True, multiple_tables=True)
df = pd.concat(df)

## Format Table

In [11]:
df_export = df.copy()

#Rename Columns
df_export.columns = ['empty0', 'name', 'location', 'country', 'address', 'uci', 'donations_grants', 'sponsorship', 'registration_fees','travel_accommodation', 'fees', 'related_expenses', 'empty1', 'total']

#Shift
df_export[df_export['empty0'].notna()] = df_export[df_export['empty0'].notna()].shift(1, axis='columns')

#Add Type
df_export = df_export.reset_index(drop=True)
index_hco = df_export[df_export['name'].str.contains("OTHER, NOT INCLUDED ABOVE", na=False)].index[0]
df_export['type'] = np.where(df_export.index < index_hco, 'hcp', 'hco')
index_hco

#Remove rows which have no values
df_export = df_export.dropna(subset=['total'], how='all')
df_export = df_export.dropna(subset=['country'], how='all')
df_export = df_export.reset_index(drop=True)

#Remove Rows with "Full Name"
df_export = df_export[~df_export['name'].str.contains('Full Name')]

#Remove empty
df_export.drop(columns=['empty0', 'empty1'], inplace=True)

#Convert to Numbers
df_export = replace_in_number(df_export, 'N/A', '')
df_export = cleanup_number(df_export)
df_export = amounts_to_number(df_export)

#Remove Carinatino
df_export = remove_carination(df_export, " ")

#basic string conversion
df_export = basic_string_conversion(df_export)

#Add PLZ
df_export = add_plz(df_export)

#Export
export_list(df_export, 'bristol')


saved


In [48]:
#write_to_csv(df_export, 'tmp.csv')
#write_to_excel(df_export, 'tmp.xlsx', True)